In [1]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast , TFBertModel


class SpamClassifier(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SpamClassifier, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained('bert-base-uncased')
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        bert_output = self.bert(input_ids=inputs['input_ids'], 
                                attention_mask=inputs['attention_mask'], 
                                token_type_ids=inputs['token_type_ids'])[1]
        return self.dense(bert_output)


with open('Sentiment_analysis.json', 'r') as json_file:
    model_json = json_file.read()

In [2]:
model = tf.keras.models.model_from_json(model_json, custom_objects={'SpamClassifier': SpamClassifier})


dummy_inputs = {
    'input_ids': tf.zeros((1, 128), dtype=tf.int32),
    'attention_mask': tf.zeros((1, 128), dtype=tf.int32),
    'token_type_ids': tf.zeros((1, 128), dtype=tf.int32)
}
model(dummy_inputs)


model.load_weights('Sentiment_analysis_weights.h5')


tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


new_data = ["Ok lar... Joking wif u oni...", "This is a negative review."]


encoded_inputs = tokenizer(new_data, padding=True, truncation=True, return_tensors='tf', max_length=128)


inputs = {
    'input_ids': encoded_inputs['input_ids'],
    'attention_mask': encoded_inputs['attention_mask'],
    'token_type_ids': encoded_inputs.get('token_type_ids', tf.zeros_like(encoded_inputs['input_ids']))
}

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
predictions = model(inputs)
predicted_probs = tf.nn.sigmoid(predictions).numpy()  # Convert logits to probabilities
predicted_labels = (predicted_probs > 0.6).astype(int)  # Convert probabilities to binary labels


for text, prob, label in zip(new_data, predicted_probs, predicted_labels):
    print(f"Text: {text}")
    print(f"Probability: {prob[0]:.4f}")
    print(f"Predicted Label: {'spam' if label == 1 else 'ham'}")
    print()
    print()

Text: Ok lar... Joking wif u oni...
Probability: 0.5006
Predicted Label: ham


Text: This is a negative review.
Probability: 0.5041
Predicted Label: ham


